This notebook reads the database of job adverts and builds a new table with the preprocessed data to facilitate further exploitation.

In [2]:
import os, json
import pandas as pd
import timeit
import sqlalchemy as alch

In [3]:
# Open the jobs db
engine = alch.create_engine('sqlite:///jobs.db', echo=False)

In [4]:
# Open the new preprocessed jobs table
engine_new = alch.create_engine('sqlite:///jobs_preprocessed.db', echo=False)

In [5]:
with engine.connect() as con:

    rs = con.execute('SELECT COUNT(*) FROM "jobs.db"')

    for row in rs:
        count = row[0]

In [6]:
conn = engine_new.connect()


In [7]:
metadata = alch.MetaData()
jobs = alch.Table('jobs', metadata,
              alch.Column('jobid', alch.Integer, primary_key=True),
              alch.Column('jobTitle', alch.Text),
              alch.Column('jobDescription', alch.Text)
             )
metadata.create_all(engine_new)

In [8]:
%%time
# NLTK Stop words
from nltk.corpus import stopwords
import spacy, re, gensim

stop_words = stopwords.words('english')
stop_words.extend(['use', 'work', 'role', 'exciting','like', 'candidate',
                   'skill','experience','join','team','business','career','hourly','part','time','hourly','hour','pay','day'])

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

nlp = spacy.load('en', disable=['parser', 'ner'])


Wall time: 1.72 s


In [9]:
%%time
i=0
f=0

with engine.connect() as con:

    rs = con.execute('SELECT jobid, jobTitle, jobDescription FROM "jobs.db"')

    for row in rs:
        
        try:
            temp = re.sub("<.*?>|&.*?;", " ", row[2])
            data_words = sent_to_words([temp])
            data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
            
            entry = {'jobid': row[0],'jobTitle': row[1], 'jobDescription':data_lemmatized[0]}
            conn.execute(jobs.insert(), entry)
         
            
        except:
            f +=1

        i+=1        
        if i %10000 == 0:
            print(i,"entries parsed")

10000 entries parsed
20000 entries parsed
30000 entries parsed
40000 entries parsed
50000 entries parsed
60000 entries parsed
70000 entries parsed
80000 entries parsed
90000 entries parsed
100000 entries parsed
110000 entries parsed
120000 entries parsed
130000 entries parsed
140000 entries parsed
150000 entries parsed
160000 entries parsed
170000 entries parsed
180000 entries parsed
190000 entries parsed
200000 entries parsed
210000 entries parsed
220000 entries parsed
230000 entries parsed
240000 entries parsed
250000 entries parsed
260000 entries parsed
270000 entries parsed
280000 entries parsed
290000 entries parsed
300000 entries parsed
310000 entries parsed
320000 entries parsed
330000 entries parsed
340000 entries parsed
350000 entries parsed
360000 entries parsed
370000 entries parsed
380000 entries parsed
390000 entries parsed
400000 entries parsed
410000 entries parsed
420000 entries parsed
430000 entries parsed
440000 entries parsed
450000 entries parsed
460000 entries pars